In [10]:
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader

dictionary_url = ("https://raw.githubusercontent.com/undertheseanlp/dictionary/master/dictionary/words.txt")
tokenizer_dataset = load_dataset("json", data_files=dictionary_url, split="train")

print(tokenizer_dataset[-1]["text"])

from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer
from tokenizers.normalizers import Replace, Lowercase, NFC
from tokenizers.pre_tokenizers import Whitespace, Metaspace
from tokenizers.models import BPE, WordLevel, Unigram
from tokenizers.trainers import BpeTrainer, WordLevelTrainer, UnigramTrainer

# tokenizer = Tokenizer(WordLevel(unk_token="<|unk|>"))
tokenizer = Tokenizer(BPE())

# Normalization Form C (NFC): Characters are decomposed and then re-composed by canonical equivalence
tokenizer.normalizer = normalizers.NFC() # "â" (U+00E2) -> "a" (U+0061) + "^" (U+0302) -> "â" (U+00E2)

# Using a pre-tokenizer will ensure no token is bigger than a word returned by the pre-tokenizer.
# tokenizer.pre_tokenizer = Whitespace()
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

# The most important part is to give the special_tokens we plan to use later on.
# trainer = WordLevelTrainer(special_tokens=["<|unk|>", "<|endoftext|>"], vocab_size=200_000)
trainer = BpeTrainer(special_tokens=["<|endoftext|>"], initial_alphabet=pre_tokenizers.ByteLevel.alphabet()) # vocab_size= 50_000, limit_alphabet = 256, min_frequency=2,

# The API to train our tokenizer will require an iterator of batch of texts, for instance a list of list of texts:

# To avoid loading everything into memory, we define a Python iterator.
def batch_iterator(batch_size=1_000, field="text"):
    for i in range(0, len(tokenizer_dataset), batch_size):
        yield tokenizer_dataset[i : i + batch_size][field]
        
from tqdm import tqdm
iterator = tqdm(batch_iterator(), desc="Training tokenizer", unit=" batch")

''' WARNING '''
tokenizer.train_from_iterator(iterator, trainer=trainer)
''' WARNING '''

# from tokenizers.processors import TemplateProcessing
# tokenizer.post_processor = TemplateProcessing(
#     single="$A <|endoftext|>",
#     pair="$A <|endoftext|> $B:1 <|endoftext|>:1",
#     special_tokens=[
#         ("<|endoftext|>", tokenizer.token_to_id("<|endoftext|>")),
#     ],
# )

tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

from transformers import GPT2TokenizerFast

new_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

new_tokenizer.save_pretrained("/llm.c/dev/data/bpe_tokenizer_20k_gpt-2")

main_tokenizer = GPT2TokenizerFast.from_pretrained("/llm.c/dev/data/bpe_tokenizer_20k_gpt-2")

output = main_tokenizer.encode("Theo chuyeen gia kinh tếe Hồ Quốc Tuaấn, Giảng viên cao cấp Đại học Bristol, Anh, "
                              "giới trẻ lười kết hôn, sinh con một phần đến từ áp lực ngày càng gia tăng khi giá bất động sản liên tục tăng lên. "
                              "“30 tuổi, không đủ giàu để mua chung cư, nhưng cũng chẳng đủ nghèo để mua nhà ở xã hội. "
                              "Áp lực này là vấn đề không chỉ ở Việt Nam, mà còn ở nhiều nước trên thế giới”, ông nói. ")

print(output)

print(main_tokenizer.decode(output))

閣下


Training tokenizer: 80 batch [00:00, 118.03 batch/s]


[449, 495, 279, 315, 13764, 770, 883, 1071, 69, 3248, 3367, 355, 513, 506, 12, 991, 497, 759, 828, 1253, 5223, 451, 558, 1129, 83, 602, 76, 12, 4664, 12, 1154, 2458, 6784, 1493, 1662, 12, 550, 593, 639, 1789, 1026, 718, 1936, 777, 1090, 2233, 770, 3079, 1788, 852, 942, 682, 871, 1719, 1471, 3079, 1213, 14, 221, 4276, 251, 19, 16, 2353, 12, 536, 3193, 3355, 1778, 2992, 1273, 1657, 12, 5545, 1254, 796, 3193, 4188, 1778, 2992, 641, 1211, 1699, 884, 14, 5594, 80, 777, 2289, 851, 2285, 1143, 536, 967, 1211, 3364, 1379, 12, 1098, 1185, 1211, 2287, 524, 1786, 720, 1154, 4276, 252, 12, 1018, 787, 14, 221]
Theo chuyeen gia kinh tếe Hồ Quốc Tuaấn, Giảng viên cao cấp Đại học Bristol, Anh, giới trẻ lười kết hôn, sinh con một phần đến từ áp lực ngày càng gia tăng khi giá bất động sản liên tục tăng lên. “30 tuổi, không đủ giàu để mua chung cư, nhưng cũng chẳng đủ nghèo để mua nhà ở xã hội. Áp lực này là vấn đề không chỉ ở Việt Nam, mà còn ở nhiều nước trên thế giới”, ông nói. 


In [4]:
import shutil
import os

# Get the default cache directory
cache_dir = os.path.expanduser("~/.cache/huggingface/datasets")

# Remove the cache directory and all its contents
shutil.rmtree(cache_dir)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\AD/.cache/huggingface/datasets'

In [8]:
import os
 
cpuCount = os.cpu_count()
print("Number of CPUs in the system:", cpuCount)

Number of CPUs in the system: 12
